In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print('x_train:', x_train.shape)
K = len(np.unique(y_train)) # Classes
Ntr = x_train.shape[0]
Nte = x_test.shape[0]
Din = 3072 # CIFAR10
# Din = 784 # MINIST
# Normalize pixel values
# x_train, x_test = x_train / 255.0, x_test / 255.0
mean_image = np.mean(x_train, axis=0)
x_train = x_train - mean_image
x_test = x_test - mean_image
y_train = tf.keras.utils.to_categorical(y_train, num_classes=K)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=K)
x_train = np.reshape(x_train,(Ntr,Din))
x_test = np.reshape(x_test,(Nte,Din))
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print('x_train:', x_train.shape)
std=1e-6
H=200
w1 = std*np.random.randn(Din, H)
w2 = std*np.random.randn(H, K)
b1 = np.zeros(H)
b2 = np.zeros(K)
print("w1:", w1.shape)
print("b1:", b1.shape)
batch_size = 500
iterations =300
lr =1.4e-2
lr_decay=0.999
reg =5e-6
loss_history = []
train_acc_history = []
val_acc_history = []
seed = 0

for t in range(iterations):
    rng = np.random.default_rng(seed=seed)
    indices = np.arange(Ntr)
    rng.shuffle(indices)
    x_train_shuffle=x_train[indices]
    y_train_shuffle=y_train[indices]

    for batch_ind in range(Ntr//batch_size):

        # Forward pass
        x_tr=x_train_shuffle[(batch_ind*batch_size):(batch_ind+1)*batch_size]
        y_tr=y_train_shuffle[(batch_ind*batch_size):(batch_ind+1)*batch_size]
        h=1.0/(1.0 + np.exp(-np.matmul(x_tr,w1)-b1))
        y_pre=np.matmul(h,w2) + b2
        loss=(1/batch_size)*np.square(y_pre-y_tr).sum()  + reg*(np.sum(w1*w1)+np.sum(w2*w2))
        loss_history.append(loss)
        
    
        # Backward pass
        dy_pre =(1/batch_size)*2.0*(y_pre-y_tr)
        dw2= np.matmul(h.T,dy_pre) +reg*w2
        db2=dy_pre.sum(axis=0)
        dh=np.matmul(dy_pre,w2.T)
        dw1= np.matmul(x_tr.T,dh*h*(1-h)) +reg*w1
        db1=(dh*h*(1-h)).sum(axis=0)
        w1-=lr*dw1
        w2-=lr*dw2
        b1-=lr*db1
        b2-=lr*db2
        lr*=lr_decay
    seed+=1

    h_s_=1.0/(1.0 + np.exp(-np.matmul(x_train_shuffle,w1)-b1))
    y_pre_s=np.matmul(h_s,w2) + b2
    train_acc=1.0 -(1/Ntr)*np.count_nonzero((np.argmax(y_train_shuffle,axis=1) - np.argmax(y_pre_s,axis=1)))
    train_acc_history.append(train_acc)
    if t % 10==0:
        print('iteration %d : loss %f : accuracy %f '% (t,loss,train_acc))


plt.plot(loss_history)
# Printing accuracies and displaying w as images
h=1.0/(1.0 + np.exp(-np.matmul(x_train,w1)-b1))
y_pre=np.matmul(h,w2) + b2
train_acc= 1.0 - (1/Ntr)*np.count_nonzero((np.argmax(y_train,axis=1) - np.argmax(y_pre,axis=1)))
print('Training accuracy : %f'% (train_acc))

h_test=1.0/(1.0 + np.exp(-np.matmul(x_test,w1)-b1))
y_test_pre=np.matmul(h_test,w2) + b2
test_acc= 1.0 - (1/Nte)*np.count_nonzero((np.argmax(y_test,axis=1) - np.argmax(y_test_pre,axis=1)))
print('Testing accuracy : %f'% (test_acc))

